# Mutli-Run Evaluation

In [1]:
import sys
import os
# Make sure the root or 03_optimization folder is in the path
sys.path.append(os.path.abspath('../'))   # adjust path if needed
sys.path.append(os.path.abspath('../..')) 

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from pathlib import Path
import re

from evaluation.evaluator import Evaluator
from evaluation.multi_run_evaluator import MultiRunEvaluator

In [5]:
#time_of_executions = ["2025-08-14_13-01", "2025-08-14_15-34"] # 3 days all models (ideal weird) => gt_freq=15min
#time_of_executions = ["2025-08-14_15-59", "2025-08-14_16-04"] # 2 days all models (gt_freq=60min)

#time_of_executions = ["2025-08-14_17-20"] # 3 days all models gt_freq=60min

#time_of_executions = ["2025-08-15_11-55", "2025-08-15_15-54", "2025-08-16_08-40", "2025-08-16_08-43", "2025-08-16_10-43", "2025-08-17_18-45"] # 7 days, mpc_f=1h, gt_f=1min, 4 models, including fixed rule-based implementation electricity_price_estimation3.csv
#time_of_executions = ["2025-08-16_12-48"] # First 15min resolution. SFH4, gt_freq=1min, 1 week, all models

#time_of_executions = ["2025-08-16_13-04", "2025-08-16_13-11"] # detection of wrong rule-base implementation
#time_of_executions = ["2025-08-16_13-16"] # with adjusted prices DImport=5, => Interval and MPCProb many bad solver results
#time_of_executions = ["2025-08-16_14-32"] # original electricity_price_estimation.csv
#time_of_executions = ["2025-08-16_14-52", "2025-08-17_08-58"] # original electricity_price_estimation.csv but 1 full week for SFH12, SFH16, SFH30 (and load result from SFH4) (mpc=1h, gt=1min)

#time_of_executions = ["2025-08-16_12-40", "2025-08-16_12-41"] # rule based comparison for different frequencies

#time_of_executions = ["2025-08-17_18-58"] # completely static prices
#time_of_executions = ["2025-08-17_19-12"] # full tou tariffs

# Checking det constraint e_end=e0
#time_of_executions = ["2025-08-18_10-39"]  # Without end constraint and electricity_price_estimation4.csv
#time_of_executions = ["2025-08-18_10-42"]  # With end constraint    and electricity_price_estimation4.csv


#time_of_executions = ["2025-08-18_09-23", "2025-08-18_09-24"] # electricity_price_estimation3.csv => rule-based strong => Also bad solver result
#time_of_executions = ["2025-08-18_09-41", "2025-08-18_09-42"] # electricity_price_estimation4.csv => ???   => Also bad solver result

# FIXED BAD SOLVER STATE WHEN e not in [e_min, e_max]
#time_of_executions = ["2025-08-18_10-14", "2025-08-18_10-15"] # redone electricity_price_estimationDImp5.csv 

# Redo estimation3 and 4 with fixed solver results
#time_of_executions = ["2025-08-18_11-17", "2025-08-18_11-18"] # electricity_price_estimation3.csv => rule-based strong 
#time_of_executions = ["2025-08-18_11-19", "2025-08-18_11-20"] # electricity_price_estimation4.csv => ???   

#time_of_executions = ["2025-08-18_12-37", "2025-08-18_12-38", "2025-08-18_12-39"]  # electricity_price_estimation5.csv

# Check other TOU tarif with greater difference
#time_of_executions = ["2025-08-18_13-59", "2025-08-18_14-00"]  # using tou_prices2.csv

# Redone electricity_price_estimation3.csv with original config.json file
#time_of_executions = ["2025-08-18_14-09", "2025-08-18_14-10"] # electricity_price_estimation3 

# Check third TOU scheme with only Export changes
#time_of_executions = ["2025-08-18_14-40", "2025-08-18_14-41"]


# Test Quadratic objective
#time_of_executions = ["2025-08-19_11-43", "2025-08-19_12-42", "2025-08-19_12-45", "2025-08-19_12-46", "2025-08-19_13-34", "2025-08-19_13-43"]  # quad_prices.csv

# Quadratic objective for SFH4 and 15min; SFH12, SFH16; SFH30(with other forecast creation time (from max))
#time_of_executions = ["2025-08-19_14-40", "2025-08-19_14-41", "2025-08-20_08-30", "2025-08-20_08-31", "2025-08-20_08-32"]

# Quadratic Objective, mpc_freq=30
#time_of_executions = ["2025-08-20_12-48", "2025-08-20_12-49", "2025-08-21_08-00"]


# Testing exponential_prices.csv 
#time_of_executions = ["2025-08-20_16-56", "2025-08-21_14-52"] # exponential_prices.csv mpc_freq=1h + DET FORECAST
#time_of_executions = ["2025-08-21_09-41", "2025-08-21_09-42", "2025-08-21_09-43", "2025-08-21_14-53"] # exponential_prices.csv mpc_freq=30min


# Check other battery configs => 13.5 kWh
#time_of_executions = ["2025-08-21_10-19", "2025-08-21_10-21", "2025-08-21_14-06"] # exponential_prices.csv, mpc_freq=60min => Bigger battery



# GIGA CALCULATION
#time_of_executions = ["2025-08-21_16-46", "2025-08-21_16-49"] # 60min-JUNE-ALL_BUILDINGS-EXPONENTIAL-13.5
#time_of_executions = ["2025-08-21_16-52", "2025-08-21_16-54"] # 60min-SEPTEMBER- ALL_BUILDINGS-EXPONENTIAL-13.5

# Check new exp prices 
#time_of_executions = ["2025-08-22_08-24"] #exponential_prices2.csv => 3 buildings to check how rule-based differs now, JUNE
#time_of_executions = ["2025-08-22_09-40", "2025-08-22_09-41", "2025-08-22_09-42"] #exponential_prices2.csv NOVEMBER



# New quad prices for grid-friendliness quad_prices2.csv
#time_of_executions = ["2025-08-22_13-36"] # June
#time_of_executions = ["2025-08-22_13-37"] # November

#time_of_executions = ["2025-08-22_13-52"] # June GTFreq=1h
#time_of_executions = ["2025-08-22_13-48"] # November GTFreq = 1h


# Testing new metrics over averaged pg
#time_of_executions = ["2025-08-22_15-52", "2025-08-22_15-53"]
#time_of_executions = ["2025-08-22_16-07","2025-08-22_16-08", "2025-08-22_16-09", "2025-08-22_16-10"] # JUNE with new metrics quad_prices2.csv gtfreq=1min
#time_of_executions = ["2025-08-22_16-11", "2025-08-22_16-12", "2025-08-22_16-13"] # November


# Runs to check mpc freq impact on new metrics in November
#time_of_executions = ["2025-08-22_16-46"] # 60MIN, BASELINE
#time_of_executions = ["2025-08-22_16-47"] # 60MIN, SMALL BATTERY
#time_of_executions = ["2025-08-22_16-49", "2025-08-22_16-50", "2025-08-22_16-51", "2025-08-22_16-52"] # 30MIN
#time_of_executions = ["2025-08-22_16-55","2025-08-22_16-56", "2025-08-22_16-57", "2025-08-22_16-59"] # 15MIN



# Maybe combine TOU with exp tariff?
#time_of_executions = ["2025-08-25_13-36", "2025-08-25_13-38", "2025-08-25_13-39", "2025-08-25_15-19"] # JUNE tou_prices.csv MPC=1h
#time_of_executions = ["2025-08-25_13-41", "2025-08-25_13-42", "2025-08-25_13-43"] # November tou_prices.csv MPC=1h


# Investigate Rule-Based Enhanced MPC for June
#time_of_executions = ["2025-08-26_09-49", "2025-08-26_09-53", "2025-08-26_09-54", "2025-08-26_10-01", "2025-08-26_09-55", "2025-08-25_13-36", "2025-08-25_13-38", "2025-08-25_13-39", "2025-08-25_15-19"]
# RBMPC from July-September
time_of_executions = ["2025-08-26_10-20", "2025-08-26_10-21"]
# RBMPC from July-September in 30min resolution
#time_of_executions = ["2025-08-26_11-06", "2025-08-26_11-08", "2025-08-26_11-09"]

# RBMPC for November
#time_of_executions = ["2025-08-26_13-37", "2025-08-26_13-38", "2025-08-25_13-41", "2025-08-25_13-42", "2025-08-25_13-43"]
# New logic that covers load if discharging
#time_of_executions = ["2025-08-26_15-14", "2025-08-26_15-19", "2025-08-26_15-20", "2025-08-25_13-41", "2025-08-25_13-42", "2025-08-25_13-43"]

# RBMPC from October-December
#time_of_executions = ["2025-08-26_16-10", "2025-08-26_16-11"] # 60min resolution
#time_of_executions = ["2025-08-26_16-22", "2025-08-26_16-23", "2025-08-26_16-24"] # 30min resolution


# First battery degradation evaluation
time_of_executions = ["2025-08-27_16-35",  "2025-08-27_16-36"]
time_of_executions = ["2025-08-27_16-41"]
time_of_executions = ["2025-08-27_17-22"]

time_of_executions = ["2025-08-27_17-33"]  # Summer 2 months
#time_of_executions = ["2025-08-27_17-36"]  # Winter 2 months
time_of_executions = ["2025-09-03_15-43", "2025-09-03_15-46"]  # 1 Week in November with different battery degradation costs (0.0 0.084, 0.09, 0.1, 0.15) f60-gt60-cX
#time_of_executions = ["2025-09-03_15-53", "2025-09-03_15-54"]  # 1 Week in November with different battery degradation costs (0.0 0.084, 0.1, 0.15) f15-gt15-cX


# GIGA CALCULATION
#time_of_executions = ["2025-08-28_13-39"]  # f60-gt60-c0
#time_of_executions = ["2025-08-29_11-24"]  # f60-gt60-c8  CHeck abgestürzt => Neu "2025-08-29_11-24"
#time_of_executions = ["2025-08-28_13-44"]  # f60-gt1-c0
time_of_executions = ["2025-08-28_13-50"]  # f60-gt1-c8

#time_of_executions = ["2025-08-28_13-46"]  # f30-gt30-c0
#time_of_executions = ["2025-08-28_13-54"]  # f30-gt30-c8
#time_of_executions = ["2025-08-28_13-56"]  # f30-gt1-c0
#time_of_executions = ["2025-08-28_13-59"]  # f30-gt1-c8

#time_of_executions = ["2025-08-28_14-02", "2025-08-28_14-03"]  # f15-gt15-c0
#time_of_executions = ["2025-08-28_14-06"]  # f15-gt15-c8
#time_of_executions = ["2025-08-28_14-04"]  # f15-gt1-c0
#time_of_executions = ["2025-08-28_14-09"]  # f15-gt1-c8



csvs = [p for t in time_of_executions for p in Path("../runs").rglob(f"logs_{t}*.csv")]

csvs = sorted(csvs,key=lambda p: int(re.search(r"SFH(\d+)", p.name).group(1)))

mre = MultiRunEvaluator(run_paths=csvs)

mre.leaderboard_mean()
#mre.leaderboard()


/srv/fh6281/GermanBuildingDate/03_optimization/evaluation/evaluator.py:27: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = (pd.read_csv(df_run, parse_dates=['timestamp']) if isinstance(df_run, (str, Path)) else df_run.copy())
/srv/fh6281/GermanBuildingDate/03_optimization/evaluation/multi_run_evaluator.py:66: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)
/srv/fh6281/GermanBuildingDate/03_optimization/evaluation/evaluator.py:27: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  self.df = (pd.read_csv(df_run, parse_dates=['timestamp']) if isinstance(df_run, (str, Path)) else df_run.copy())
/srv/fh6281/GermanBuildingDate/03_optimization/evaluation/multi_run_evaluator.py:66: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


,model,runs,buildings,freq,e_import_total,import_cost,e_export_total,export_revenue,e_throughput,e_discharged_total,e_battery_deg_costs,e_end,net_cost,net_cost_adj,net_cost_final,avg_rank
0,ideal_rule,15,15,60.0,1255.59,275.95,1210.80,131.69,2219.43,1084.30,91.08,10.01,144.27,143.60,234.68,1.00
1,rule-based,15,15,60.0,1139.61,323.93,1125.71,121.54,1170.70,575.37,48.33,0.31,202.39,202.37,250.70,2.47
2,mpc_det_rule,15,15,60.0,1343.28,312.69,1305.08,142.20,1990.64,973.19,81.75,8.05,170.48,169.95,251.70,2.53
3,ideal,15,15,60.0,1671.39,404.92,1629.33,181.42,2085.08,1018.48,85.55,10.00,223.50,222.83,308.38,4.00


In [4]:
mre.leaderboard()


🏢 Building: SFH3


,model,building,freq,t_start,t_end,solver_fails,e_import_total,import_cost,e_export_total,export_revenue,e_throughput,e_discharged_total,e_battery_deg_costs,e_end,net_cost,net_cost_adj,net_cost_final
0,ideal_rule,SFH3,30,2020-07-20 04:00:00+00:00,2020-12-20 03:59:00+00:00,30/220320 (0.01%),1053.5018,258.647322,1688.3107,235.465364,3523.7644,1723.5652,144.7795,9.4552,23.181957,22.553186,167.332686
1,rule-based,SFH3,30,2020-07-20 04:00:00+00:00,2020-12-20 03:59:00+00:00,0/220320 (0.0%),968.6843,296.143907,1657.0230,175.930760,1341.7525,659.3241,55.3832,0.0000,120.213148,120.213148,175.596348
2,mpc_det_rule,SFH3,30,2020-07-20 04:00:00+00:00,2020-12-20 03:59:00+00:00,90/220320 (0.04%),1175.1339,299.918110,1812.0865,244.726664,3285.4414,1605.4756,134.8600,12.1262,55.191446,54.385054,189.245054
3,ideal,SFH3,30,2020-07-20 04:00:00+00:00,2020-12-20 03:59:00+00:00,60/220320 (0.03%),1456.4590,379.634474,2090.7845,291.097325,3571.6947,1747.2887,146.7723,8.9702,88.537148,87.940630,234.712930



🏢 Building: SFH4


,model,building,freq,t_start,t_end,solver_fails,e_import_total,import_cost,e_export_total,export_revenue,e_throughput,e_discharged_total,e_battery_deg_costs,e_end,net_cost,net_cost_adj,net_cost_final
0,ideal_rule,SFH4,30,2020-07-20 04:00:00+00:00,2020-12-20 03:59:00+00:00,90/220320 (0.04%),1372.3209,295.246507,1731.6929,233.873885,3874.4007,1893.9872,159.0949,12.1652,61.372622,60.563637,219.658537
1,rule-based,SFH4,30,2020-07-20 04:00:00+00:00,2020-12-20 03:59:00+00:00,0/220320 (0.0%),1293.4719,354.601572,1708.6259,182.243291,1535.5593,752.4575,63.2064,3.6259,172.358281,172.117159,235.323559
2,mpc_det_rule,SFH4,30,2020-07-20 04:00:00+00:00,2020-12-20 03:59:00+00:00,120/220320 (0.05%),1451.0183,340.310246,1824.9674,241.304005,3584.2485,1756.1997,147.5208,3.4473,99.006241,98.776995,246.297795
3,ideal,SFH4,30,2020-07-20 04:00:00+00:00,2020-12-20 03:59:00+00:00,120/220320 (0.05%),1807.3718,426.959569,2167.3448,288.579629,3844.7424,1879.4586,157.8745,12.1632,138.379940,137.571087,295.445587



🏢 Building: SFH9


...

,model,building,freq,t_start,t_end,solver_fails,e_import_total,import_cost,e_export_total,export_revenue,e_throughput,e_discharged_total,e_battery_deg_costs,e_end,net_cost,net_cost_adj,net_cost_final
0,ideal_rule,SFH30,30,2020-07-20 04:00:00+00:00,2020-12-20 03:59:00+00:00,30/220320 (0.01%),577.3572,119.100020,696.4305,111.963850,2427.2783,1186.7213,99.6846,8.8057,7.136170,6.550591,106.235191
1,rule-based,SFH30,30,2020-07-20 04:00:00+00:00,2020-12-20 03:59:00+00:00,0/220320 (0.0%),529.0600,157.415100,690.8813,77.601993,746.8692,367.8908,30.9028,0.0000,79.813107,79.813107,110.715907
2,mpc_det_rule,SFH30,30,2020-07-20 04:00:00+00:00,2020-12-20 03:59:00+00:00,0/220320 (0.0%),616.6838,142.761841,742.6151,112.525517,2169.3713,1061.1969,89.1405,7.1570,30.236324,29.760384,118.900884
3,ideal,SFH30,30,2020-07-20 04:00:00+00:00,2020-12-20 03:59:00+00:00,30/220320 (0.01%),920.8026,222.030632,1037.9351,159.482302,2531.1315,1237.6776,103.9649,8.6485,62.548330,61.973205,165.938105



🏢 Building: SFH32


,model,building,freq,t_start,t_end,solver_fails,e_import_total,import_cost,e_export_total,export_revenue,e_throughput,e_discharged_total,e_battery_deg_costs,e_end,net_cost,net_cost_adj,net_cost_final
0,ideal_rule,SFH32,30,2020-07-20 04:00:00+00:00,2020-12-20 03:59:00+00:00,60/220320 (0.03%),1030.9500,219.034826,1239.2851,181.179708,3433.1001,1679.6960,141.0945,8.3606,37.855118,37.299139,178.393639
1,mpc_det_rule,SFH32,30,2020-07-20 04:00:00+00:00,2020-12-20 03:59:00+00:00,30/220320 (0.01%),1096.6252,250.163039,1311.1712,187.516533,3172.3802,1552.4415,130.4051,7.4160,62.646506,62.153342,192.558442
2,rule-based,SFH32,30,2020-07-20 04:00:00+00:00,2020-12-20 03:59:00+00:00,0/220320 (0.0%),967.5027,279.073711,1229.6272,134.809596,1184.0624,582.0718,48.8940,0.0000,144.264115,144.264115,193.158115
3,ideal,SFH32,30,2020-07-20 04:00:00+00:00,2020-12-20 03:59:00+00:00,30/220320 (0.01%),1382.9840,324.342987,1591.4549,225.640153,3432.4872,1679.4574,141.0744,8.2372,98.702834,98.155060,239.229460



🏢 Building: SFH36


,model,building,freq,t_start,t_end,solver_fails,e_import_total,import_cost,e_export_total,export_revenue,e_throughput,e_discharged_total,e_battery_deg_costs,e_end,net_cost,net_cost_adj,net_cost_final
0,ideal_rule,SFH36,30,2020-07-20 04:00:00+00:00,2020-12-20 03:59:00+00:00,150/220320 (0.07%),1472.8439,338.596783,731.8340,117.891124,2873.7033,1404.0312,117.9386,11.5937,220.705659,219.934678,337.873278
1,mpc_det_rule,SFH36,30,2020-07-20 04:00:00+00:00,2020-12-20 03:59:00+00:00,30/220320 (0.01%),1493.0092,358.910091,757.2108,113.597749,2619.1931,1279.3819,107.4681,11.5233,245.312342,244.546043,352.014143
2,rule-based,SFH36,30,2020-07-20 04:00:00+00:00,2020-12-20 03:59:00+00:00,0/220320 (0.0%),1416.5525,407.256545,730.4381,81.561165,729.9292,359.5919,30.2057,0.0000,325.695380,325.695380,355.901080
3,ideal,SFH36,30,2020-07-20 04:00:00+00:00,2020-12-20 03:59:00+00:00,60/220320 (0.03%),1925.0979,476.646114,1181.8961,181.054097,2984.8709,1458.5190,122.5156,11.5401,295.592017,294.824600,417.340200


In [ ]:
mre.df.columns

Index(['path', 'timestamp', 'model', 'freq', 'building', 't_start', 't_end',
       'total_timespan', 'import_cost', 'export_revenue', 'net_cost',
       'net_cost_adj', 'import_squared', 'export_squared', 'count', 'mss_kw2',
       'rms_kw', 'q95_abs_kw', 'pmax_import_kw', 'pmax_export_kw',
       'exceed_frac', 'exceed_hours', 'threshold_kw', 'e_end',
       'pg_import_total', 'pg_export_total', 'e_export_total',
       'e_import_total', 'steps', 'solver_fails', 'solver_fail_count',
       'solver_fail_pct', 'solver_top_errors', 'solver_status_mix'],
      dtype='object')

In [ ]:
cols_to_show=["model", "t_start", 'import_squared', 'export_squared', 'count', 'mss_kw2',
       'rms_kw', 'q95_abs_kw', 'pmax_import_kw', 'pmax_export_kw',
       'exceed_frac', 'exceed_hours', 'threshold_kw']

#mre.df[cols_to_show]
mre.leaderboard(by="mss_kw2", cols_to_show=cols_to_show)


🏢 Building: SFH4


,model,t_start,import_squared,export_squared,count,mss_kw2,rms_kw,q95_abs_kw,pmax_import_kw,pmax_export_kw,exceed_frac,exceed_hours,threshold_kw
0,ideal,2020-06-08 04:00:00+00:00,208.950321,18166.536439,10066,1.489237,1.220343,2.440282,1.278202,4.577710,0.05007,8.4,2.440282
1,rule-based,2020-06-08 04:00:00+00:00,0.000000,39858.757605,10066,3.655187,1.911854,4.618558,0.000000,6.371779,0.05007,8.4,4.618558



🏢 Building: SFH9


,model,t_start,import_squared,export_squared,count,mss_kw2,rms_kw,q95_abs_kw,pmax_import_kw,pmax_export_kw,exceed_frac,exceed_hours,threshold_kw
0,ideal,2020-06-08 04:00:00+00:00,2857.205715,24672.758944,10066,2.092736,1.446629,3.063694,3.659256,7.283195,0.05007,8.4,3.063694
1,rule-based,2020-06-08 04:00:00+00:00,296.732089,41234.777751,10066,3.702383,1.924158,5.108167,3.361099,7.735205,0.05007,8.4,5.108167



🏢 Building: SFH10


,model,t_start,import_squared,export_squared,count,mss_kw2,rms_kw,q95_abs_kw,pmax_import_kw,pmax_export_kw,exceed_frac,exceed_hours,threshold_kw
0,ideal,2020-06-08 04:00:00+00:00,1868.287487,6097.517250,10066,0.471832,0.686900,1.382799,2.297818,3.218414,0.05007,8.4,1.382799
1,rule-based,2020-06-08 04:00:00+00:00,175.238891,11544.010963,10066,1.012190,1.006077,2.821820,1.076365,4.655191,0.05007,8.4,2.821820



🏢 Building: SFH18


,model,t_start,import_squared,export_squared,count,mss_kw2,rms_kw,q95_abs_kw,pmax_import_kw,pmax_export_kw,exceed_frac,exceed_hours,threshold_kw
0,ideal,2020-06-08 04:00:00+00:00,1116.820986,3864.939325,10066,0.321001,0.566570,1.045906,2.872928,2.680746,0.05007,8.4,1.045906
1,rule-based,2020-06-08 04:00:00+00:00,5.737140,7404.232067,10066,0.673262,0.820525,2.143820,0.205814,3.145892,0.05007,8.4,2.143820


In [ ]:
mre.pivot()

model,ideal,rule-based
freq,60,60
building,,
SFH10,132.763412,195.320831
SFH18,83.029339,123.499487
SFH4,306.258113,664.312627
SFH9,458.832744,692.191831


In [ ]:
mre.failure_table()

,model,building,freq,t_start,t_end,steps,solver_fail_count,solver_fail_pct,solver_status_mix,solver_top_errors,path
0,ideal,SFH9,60,2020-06-08 04:00:00+00:00,2020-06-15 03:59:00+00:00,10080,60,0.005952,error1:60,fallback_decision:60,../runs/2025-08-22_13-36/SFH9/logs_2025-08-22_...
1,rule-based,SFH4,60,2020-06-08 04:00:00+00:00,2020-06-15 03:59:00+00:00,10080,0,0.000000,,,../runs/2025-08-22_13-36/SFH4/logs_2025-08-22_...
2,ideal,SFH4,60,2020-06-08 04:00:00+00:00,2020-06-15 03:59:00+00:00,10080,0,0.000000,,,../runs/2025-08-22_13-36/SFH4/logs_2025-08-22_...
3,rule-based,SFH9,60,2020-06-08 04:00:00+00:00,2020-06-15 03:59:00+00:00,10080,0,0.000000,,,../runs/2025-08-22_13-36/SFH9/logs_2025-08-22_...
4,ideal,SFH10,60,2020-06-08 04:00:00+00:00,2020-06-15 03:59:00+00:00,10080,0,0.000000,,,../runs/2025-08-22_13-36/SFH10/logs_2025-08-22...
5,rule-based,SFH10,60,2020-06-08 04:00:00+00:00,2020-06-15 03:59:00+00:00,10080,0,0.000000,,,../runs/2025-08-22_13-36/SFH10/logs_2025-08-22...
6,rule-based,SFH18,60,2020-06-08 04:00:00+00:00,2020-06-15 03:59:00+00:00,10080,0,0.000000,,,../runs/2025-08-22_13-36/SFH18/logs_2025-08-22...
7,ideal,SFH18,60,2020-06-08 04:00:00+00:00,2020-06-15 03:59:00+00:00,10080,0,0.000000,,,../runs/2025-08-22_13-36/SFH18/logs_2025-08-22...


In [ ]:
# dtype error => solver error
time_of_executions = ["2025-08-26_10-20", "2025-08-26_10-21"]
csvs = [p for t in time_of_executions for p in Path("../runs").rglob(f"logs_{t}*.csv")]
csvs[1]
df = pd.read_csv(csvs[5])
cols = df.columns
print("Column #9 is:", cols[9])
cols

## Test the evaluation class

In [ ]:
run_path = Path('../runs/logs_2025-08-11_08-40_op-interval_freq-60_building-SFH3.csv')
run_path_rb = Path('../runs/logs_2025-08-11_08-40_op-rule-based_freq-60_building-SFH3.csv')

prices_path = Path('../../01_data/price_data/electricity_price_estimation.csv')
prices_path_const = Path('../../01_data/price_data/const_prices_testing.csv')
ev1 = Evaluator(run_path, prices_path)
ev1.get_costs()


KeyboardInterrupt



In [ ]:
ev2 = Evaluator(run_path_rb, prices_path)
ev2.get_costs()

{'import_cost': 0.0,
 'export_revenue': 2.186811788658414,
 'net_cost': -2.186811788658414}

In [ ]:
ev1.get_df()

,action,pb_low,pb_high,pg,gt,soe_now,soe_new,import_price,export_price,costs_buy,costs_sell,cashflow
timestamp,,,,,,,,,,,,
2020-05-13 07:00:00+00:00,-0.322137,-0.322137,-1.201852e-07,-2.511872,-2.834009,4.000000,4.076507,0.1139,0.0465,0.0,0.116802,-0.116802
2020-05-13 07:15:00+00:00,-0.322137,-0.322137,-1.201852e-07,-2.557424,-2.879560,4.076507,4.153015,0.1139,0.0465,0.0,0.118920,-0.118920
2020-05-13 07:30:00+00:00,-0.322137,-0.322137,-1.201852e-07,-2.854330,-3.176466,4.153015,4.229522,0.1139,0.0465,0.0,0.132726,-0.132726
2020-05-13 07:45:00+00:00,-0.322137,-0.322137,-1.201852e-07,-3.387559,-3.709695,4.229522,4.306030,0.1139,0.0465,0.0,0.157521,-0.157521
2020-05-13 08:00:00+00:00,0.655249,0.655249,1.088511e+00,-4.045914,-3.390665,4.306030,4.133596,0.1138,0.0465,0.0,0.188135,-0.188135
2020-05-13 08:15:00+00:00,0.655249,0.655249,1.088511e+00,-4.228103,-3.572853,4.133596,3.961162,0.1138,0.0465,0.0,0.196607,-0.196607
2020-05-13 08:30:00+00:00,0.655249,0.655249,1.088511e+00,-5.395369,-4.740119,3.961162,3.788728,0.1138,0.0465,0.0,0.250885,-0.250885
2020-05-13 08:45:00+00:00,0.655249,0.655249,1.088511e+00,-5.698039,-5.042790,3.788728,3.616294,0.1138,0.0465,0.0,0.264959,-0.264959
2020-05-13 09:00:00+00:00,-0.556352,-0.556352,-1.791756e-06,-4.561881,-5.118234,3.616294,3.748427,0.1094,0.0448,0.0,0.204372,-0.204372


In [ ]:
ev1.plot_battery_soe(max_soe=8.0)

In [ ]:
ev2.plot_battery_soe(max_soe=8.0)

## Test Multi-Run Evaluator

In [ ]:
path1 = Path('../runs/logs_2025-08-11_08-40_op-interval_freq-60_building-SFH3.csv')
path2 = Path('../runs/logs_2025-08-11_08-40_op-rule-based_freq-60_building-SFH3.csv')

path_list = [path1, path2]

mre = MultiRunEvaluator(path_list)

self.run_paths: [WindowsPath('../runs/logs_2025-08-11_08-40_op-interval_freq-60_building-SFH3.csv'), WindowsPath('../runs/logs_2025-08-11_08-40_op-rule-based_freq-60_building-SFH3.csv')]


In [ ]:
mre.leaderboard(by="net_cost")

,model,building,freq,t_start,t_end,net_cost
0,interval,SFH3,60,2020-05-13 07:00:00+00:00,2020-05-13 09:45:00+00:00,-2.344862
1,rule-based,SFH3,60,2020-05-13 07:00:00+00:00,2020-05-13 09:45:00+00:00,-2.186812


In [ ]:
mre.pivot()

model,interval,rule-based
freq,60,60
building,,
SFH3,-2.344862,-2.186812
